# Floresta Aleatória Classificadora

## Introdução

Este notebook visa fazer a otimização de **hiperparâmetros**, de estratégia de **normalização**, de **seleção de atributos** e de **redução de dimensionalidade**. A seleção de atributos é realizada por *Recursive Feature Elimination* (**RFE**, ou Eliminação Recursiva de Atributos em português), a otimização testa a ausência ou não dessa estratégia. A redução de dimensionalidade é realizada por *Principal Component Analysis* (**PCA**, ou Análise de Componenetes Principais em português), é testada sua ausência também. Para normalização, são testadas a ausência, ou a normalização **padrão** ou a pelos **mínimos e máximos**. A otimização é feita usando o módulo **optuna**.

## Modelo

O modelo otimizado é uma Floresta Aleatória classificadora. Os hiperparâmetros otimizados são:

- `n_estimators`: número de Florestas de Decisão. Testados no **intervalo de 5 até 300**;
    
- `criterion`: critério usado para medir a qualidade de um split. Testado entre **'entropy'**, **'log_loss'** e **'gini'**;
    
- `min_samples_split`: mínimo de exemplos por split. Testado no **intervalo de 2 até 20**;

- `max_features`: número de atributos a serem considerados quando procurando por um split. Testado no **intervalo entre 0.1 e 1**, sendo que calcula o número de atributos considerados da seguinte forma: max(1, int(max_features * n_features_in_));

- `min_samples_leaf`: mínimo de exemplos por folha. Testado no **intervalo de 1 até 20**;

- `class_weight`: pesos entre as classes. Usado para dados desbalanceados. Testado entre **'balanced'**, que considera o desbalanço dos dados para toda a floresta e balanceia com base na proporção entre as classes, e **'balanced_subsample'**, que considera o desbalanço dos dados entregues a cada árvore.

## Carrega os dados

In [1]:
import pandas as pd

# Dados de treino
X_treino = pd.read_csv('../Dados/dados_tratados/X_treino.csv')
y_treino = pd.read_csv('../Dados/dados_tratados/y_treino.csv').values.ravel()

## Define algumas variáveis úteis

In [2]:
# Parâmetros
SEMENTE_ALEATORIA = 9 # Random state
num_max_atributos = len(X_treino.columns) - 1 # Para RFE

X_treino = X_treino.values

## Definição de funções para o optuna

O código abaixo define a função que cria uma instância do modelo. É nela que o pipeline é montado de acordo com as sugestões definidas pelo optuna.

In [3]:
from sklearn.ensemble import RandomForestClassifier # algoritmo de aprendizado de máquina
from sklearn.preprocessing import StandardScaler, MinMaxScaler # normalização
from sklearn.feature_selection import RFE # seleção de atributos
from sklearn.decomposition import PCA # redução de dimensionalidade
from sklearn.pipeline import make_pipeline

def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.

    """
    parametros = {
        "n_estimators": trial.suggest_int("num_arvores", 5, 300, log=True),
        "criterion": trial.suggest_categorical("criterio", ['entropy', 'log_loss', 'gini']),
        "min_samples_split": trial.suggest_int("min_exemplos_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_exemplos_folha", 1, 20),
        "max_features": trial.suggest_float("num_max_atributos", 0.1, 1),
        "class_weight": trial.suggest_categorical("balanceamento", ["balanced", "balanced_subsample"]), # Dados desbalanceados
        "n_jobs": 1,
        "bootstrap": True,
        "random_state": SEMENTE_ALEATORIA,
    }

    normalizacao = trial.suggest_categorical("normalizacao", ["none", "standard", "minmax"])
    usar_pca = trial.suggest_categorical("PCA", [True, False])
    usar_rfe = trial.suggest_categorical("RFE", [True, False])

    num_atributos = trial.suggest_int("num_atributos", 2, num_max_atributos) # para RFE
    num_dimensoes = trial.suggest_int("num_dimensoes", 2, num_atributos) # para PCA
        
    
    steps = []

    if normalizacao == "standard":
        steps.append(("normalizador", StandardScaler()))
    elif normalizacao == "minmax":
        steps.append(("normalizador", MinMaxScaler()))

    if usar_rfe:
        steps.append(("rfe", RFE(RandomForestClassifier(**parametros), n_features_to_select=num_atributos)))

    if usar_pca:
        steps.append(("pca", PCA(n_components=num_dimensoes, random_state=SEMENTE_ALEATORIA)))

    steps.append(("modelo", RandomForestClassifier(**parametros)))

    modelo = make_pipeline(*[s[1] for s in steps])

    return modelo

O código abaixo define a função que estima o desempenho do modelo de cada trial. Essa é a função que terá valor de retorno maximizado pelo optuna.

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
import numpy as np

f2_score = make_scorer(fbeta_score, beta=2)

def funcao_objetivo(trial, X, y, num_folds):
    """Função objetivo do optuna.

    Faz validação cruzada estratificada com métrica F2.

    """
    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring=f2_score,
        cv=num_folds,
    )

    return np.mean(metricas)

## Otimização com optuna

O código abaixo cria ou carrega um 'estudo' a depender da existência dele. O 'estudo' contém informações sobre cada trial. A variável objeto_de_estudo acessa esse 'estudo'. 

In [5]:
from optuna import create_study

NOME_DO_ESTUDO = "floresta_aleatoria_class_optuna"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-10-30 17:37:19,933] A new study created in RDB with name: floresta_aleatoria_class_optuna


O código abaixo faz a otimização com optuna.

In [6]:
# Parâmetros
NUM_FOLDS = 10 # Validação cruzada
NUM_TENTATIVAS = 200 # Quantidade de trials

def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

# Faz a otimização
objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2025-10-30 17:37:21,113] Trial 0 finished with value: 0.8568091303005578 and parameters: {'num_arvores': 9, 'criterio': 'entropy', 'min_exemplos_split': 4, 'min_exemplos_folha': 14, 'num_max_atributos': 0.40085608383442484, 'balanceamento': 'balanced', 'normalizacao': 'none', 'PCA': True, 'RFE': False, 'num_atributos': 9, 'num_dimensoes': 4}. Best is trial 0 with value: 0.8568091303005578.
[I 2025-10-30 17:37:41,357] Trial 1 finished with value: 0.8743033090424074 and parameters: {'num_arvores': 102, 'criterio': 'entropy', 'min_exemplos_split': 15, 'min_exemplos_folha': 1, 'num_max_atributos': 0.3825076887545841, 'balanceamento': 'balanced_subsample', 'normalizacao': 'minmax', 'PCA': False, 'RFE': True, 'num_atributos': 2, 'num_dimensoes': 2}. Best is trial 1 with value: 0.8743033090424074.
[I 2025-10-30 17:37:43,011] Trial 2 finished with value: 0.8926904355020288 and parameters: {'num_arvores': 23, 'criterio': 'gini', 'min_exemplos_split': 5, 'min_exemplos_folha': 7, 'num_max_atri

## DataFrame dos trials

O código abaixo mostra o DataFrame contruído pelo objeto de estudo do optuna contendo todos os parâmetros de cada trial.

In [7]:
df_analysis = objeto_de_estudo.trials_dataframe()

df_analysis

,number,value,datetime_start,datetime_complete,duration,params_PCA,params_RFE,params_balanceamento,params_criterio,params_min_exemplos_folha,params_min_exemplos_split,params_normalizacao,params_num_arvores,params_num_atributos,params_num_dimensoes,params_num_max_atributos,state
0,0,0.856809,2025-10-30 17:37:19.950936,2025-10-30 17:37:20.906146,0 days 00:00:00.955210,True,False,balanced,entropy,14,4,none,9,9,4,0.400856,COMPLETE
1,1,0.874303,2025-10-30 17:37:21.123043,2025-10-30 17:37:41.205335,0 days 00:00:20.082292,False,True,balanced_subsample,entropy,1,15,minmax,102,2,2,0.382508,COMPLETE
2,2,0.892690,2025-10-30 17:37:41.367929,2025-10-30 17:37:42.908388,0 days 00:00:01.540459,False,False,balanced_subsample,gini,7,5,minmax,23,7,4,0.450876,COMPLETE
3,3,0.847889,2025-10-30 17:37:43.022383,2025-10-30 17:37:49.304898,0 days 00:00:06.282515,True,True,balanced,gini,18,20,minmax,58,3,2,0.107913,COMPLETE
4,4,0.855968,2025-10-30 17:37:49.368427,2025-10-30 17:37:56.432078,0 days 00:00:07.063651,True,False,balanced_subsample,entropy,10,15,minmax,257,7,5,0.953701,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,0.886290,2025-10-30 17:47:35.170458,2025-10-30 17:47:38.104596,0 days 00:00:02.934138,False,True,balanced,log_loss,10,11,standard,52,9,8,0.488698,COMPLETE
196,196,0.824728,2025-10-30 17:47:38.345070,2025-10-30 17:47:40.120146,0 days 00:00:01.775076,True,True,balanced,log_loss,12,11,standard,44,9,8,0.243047,COMPLETE
197,197,0.895559,2025-10-30 17:47:40.194244,2025-10-30 17:47:43.752505,0 days 00:00:03.558261,False,True,balanced,log_loss,12,11,standard,60,9,7,0.531277,COMPLETE
198,198,0.872754,2025-10-30 17:47:43.823726,2025-10-30 17:47:45.878568,0 days 00:00:02.054842,False,True,balanced,log_loss,20,11,standard,41,9,8,0.502202,COMPLETE


## Melhor modelo

O código abaixo mostra o melhor modelo encontrado pelo optuna e seus parâmetros. Note que os parâmetros 'num_atributos' e 'num_dimensoes' só influenciam caso 'RFE' e 'PCA' sejam True respectivamente, pois são seus respectivos hiperparâmetros.

In [8]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 75
Parâmetros do melhor trial: {'num_arvores': 52, 'criterio': 'log_loss', 'min_exemplos_split': 7, 'min_exemplos_folha': 12, 'num_max_atributos': 0.4805539918242229, 'balanceamento': 'balanced', 'normalizacao': 'minmax', 'PCA': False, 'RFE': False, 'num_atributos': 7, 'num_dimensoes': 3}


Este notebook se encerra aqui. O modelo será usado no notebook chamado 'Resultados_Discussão'.

## Referências

[1] Documentação do optuna. Acesso em: 31/10/2025. Disponível em: https://optuna.readthedocs.io/en/stable/ 

[2] Página da Floresta Aleatória classificadora da documentação do scikit-learn. Acesso em: 31/10/2025. Disponível em: https://scikit-learn.org/1.5/modules/generated/sklearn.ensemble.RandomForestClassifier.html

[3] Sperat, Walter. *Using optuna with sklearn the right way*. Acesso em 31/10/2025. Disponível em: https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451